In [6]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../..")
import torch

from xlstm.blocks.slstm.cell import sLSTMCell, sLSTMCellConfig
from xlstm.blocks.slstm.block import sLSTMBlock, sLSTMBlockConfig
from xlstm.blocks.slstm.layer import sLSTMLayer, sLSTMLayerConfig
from xlstm.components.feedforward import FeedForwardConfig

backend = "vanilla" #"cuda" if torch.cuda.is_available() else "vanilla"
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
cell = sLSTMCell(sLSTMCellConfig(hidden_size=128, num_heads=2, backend=backend, dtype="bfloat16", function="slstm", enable_automatic_mixed_precision=False)).to(device)

In [8]:
res = cell(torch.zeros([4, 4, 4*128], dtype=torch.bfloat16).to(device))
print(res[0].shape, res[1].shape)

torch.Size([4, 2, 4, 64]) torch.Size([4, 4, 128])


In [4]:
block = sLSTMBlock(sLSTMBlockConfig(slstm=sLSTMLayerConfig(embedding_dim=32,
    num_heads=4, backend=backend, function="lstm", bias_init="standard", recurrent_weight_init="standard",
    _block_idx=2, _num_blocks=4, enable_automatic_mixed_precision=False), feedforward=FeedForwardConfig())).to(device).to(dtype=torch.bfloat16)

TypeError: unsupported operand type(s) for /: 'int' and 'NoneType'

In [9]:
inp = torch.ones((3, 4, 32), dtype=torch.bfloat16).to(device)
inp.requires_grad = True
res = block(inp)

NameError: name 'block' is not defined

In [6]:
res.shape

torch.Size([3, 4, 32])

In [7]:
res[1, 1].sum().backward()
# check for causality, batch interconnect
print(inp.grad[2, 1].sum(), "== 0")
print(inp.grad[0, 1].sum(), "== 0")
print(inp.grad[1, 2].sum(), "== 0")
print(inp.grad[1, 0].sum(), "!= 0")
print(inp.grad[1, 1].sum(), "!= 0")



tensor(0., device='cuda:0', dtype=torch.bfloat16) == 0
tensor(0., device='cuda:0', dtype=torch.bfloat16) == 0
tensor(0., device='cuda:0', dtype=torch.bfloat16) == 0
tensor(1.1250, device='cuda:0', dtype=torch.bfloat16) != 0
tensor(33.7500, device='cuda:0', dtype=torch.bfloat16) != 0


In [10]:
from xlstm.xlstm_block_stack import xLSTMBlockStackConfig, xLSTMBlockStack, mLSTMBlockConfig

In [11]:
bs = xLSTMBlockStack(
    xLSTMBlockStackConfig(
        slstm_block=sLSTMBlockConfig(slstm=sLSTMLayerConfig(backend=backend)),
        slstm_at="all",
        num_blocks=48,
        embedding_dim=1024,
    )
)